# Neural Bag of Words for Amazon Reviews

In [2]:
from __future__ import division
import os, sys, re, json, time, datetime, shutil
import itertools, collections
from importlib import reload
from IPython.display import display, HTML

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import pandas as pd
import tensorflow as tf
assert(tf.__version__.startswith("1.8"))

# This creates a symbolic link on python in tmp directory

# Helper libraries for 's instance
from w266_common import utils, vocabulary, tf_embed_viz, treeviz
from w266_common import patched_numpy_io

# Code for this assignment

import models

import nltk
from nltk.tokenize import word_tokenize

## Word Vectors

In [3]:
from pathlib import Path
import os.path
wordsList = np.load(os.path.join(str(Path.home()), '.kaggle/wordvectors/pretrained_glove/wordsList.npy'))
wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load(os.path.join(str(Path.home()), '.kaggle/wordvectors/pretrained_glove/wordVectors.npy'))

## Load Reviews

In [4]:
review_df = pd.read_csv('~/.kaggle/datasets/snap/amazon-fine-food-reviews/Reviews.csv', encoding='utf8')
review_df = review_df.drop(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Time', 'Summary'], axis=1)

In [5]:
# Reduce size for development
#numReviews = 100
#review_df = review_df.loc[0:numReviews-1]

## Tokenize

In [6]:
review_df['Tokens'] = review_df['Text'].apply(lambda text: word_tokenize(text.lower()))

## Train, Dev, Test data

In [7]:
# Train with 60%, , Dev: 10%, Test: 30%
train_percent = 0.6
dev_percent = 0.1
test_percent = 0.3

# Get indicies of the rows in the dataframe for training and testing
train_lower_index = 0
train_upper_index = train_lower_index + round(len(review_df)*train_percent)
dev_lower_index   = train_upper_index+1
dev_upper_index   = dev_lower_index + round(len(review_df)*dev_percent)
test_lower_index  = dev_upper_index+1
test_upper_index  = len(review_df)-1

In [8]:
maxSeqLength = 267  # Determined by EDA

In [9]:
word_ids = np.zeros([review_df.shape[0], maxSeqLength], dtype=np.int32)
word_ids_ns = np.zeros([review_df.shape[0]], dtype=np.int32)
word_ids_labels = np.zeros([review_df.shape[0]], dtype=np.int32)

In [10]:
for sentence_index, row in review_df.iterrows():
    
    word_index = 0
    
    for word in row['Tokens']:

        try:
            word_ids[sentence_index][word_index] = wordsList.index(word)
        except ValueError:
            word_ids[sentence_index][word_index] = 399999 #Vector for unkown words
        
        word_index = word_index + 1

        if word_index == maxSeqLength:
            break

    word_ids_ns[sentence_index] = word_index
    word_ids_labels[sentence_index] = row['Score']-1  # Labels are 0 based

In [11]:
def Split(lower_idx, upper_idx):
    return word_ids[lower_idx:upper_idx], word_ids_ns[lower_idx:upper_idx], word_ids_labels[lower_idx:upper_idx]

In [12]:
train_x, train_ns, train_y = Split(train_lower_index, train_upper_index)
dev_x,   dev_ns,   dev_y   = Split(dev_lower_index, dev_upper_index)
test_x,  test_ns,  test_y  = Split(test_lower_index, test_upper_index)

## Model Parameters

In [13]:
import models; reload(models)

<module 'models' from '/home/mprout/w266_final_project/NBOW_Multiclass/models.py'>

In [14]:
# Specify model hyperparameters as used by model_fn
model_params = dict(V=len(wordsList),
                    embed_dim=50,
                    hidden_dims=[25],
                    num_classes=5,      # 5 for multiclass classifier
                    encoder_type='bow',
                    lr=0.1,
                    optimizer='adagrad',
                    beta=0.01,
                    dropout_rate=0.1)  # fill this in

## Training Parameters

In [15]:
# Specify training schedule
train_params = dict(batch_size=32,
                    total_epochs=20,
                    eval_every=2)  # fill this in

In [16]:
checkpoint_dir = "/tmp/tf_bow_sst_" + datetime.datetime.now().strftime("%Y%m%d-%H%M")

if os.path.isdir(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)
#ds.vocab.write_projector_config(checkpoint_dir, "Encoder/Embedding_Layer/W_embed")

In [17]:
model = tf.estimator.Estimator(model_fn=models.classifier_model_fn,
                               params=model_params,
                               model_dir=checkpoint_dir)

print("\nTo view training (once it starts), run:\n")
print("    tensorboard --logdir='{:s}' --port 6006".format(checkpoint_dir))
print("\nThen in your browser, open: http://localhost:6006\n")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tf_bow_sst_20180720-0205', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7aebc48978>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}

To view training (once it starts), run:

    tensorboard --logdir='/tmp/tf_bow_sst_20180720-0205' --port 6006

Then in your browser, open: http://localhost:6006



In [18]:
train_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"ids": train_x, "ns": train_ns},
                    y=train_y,
                    batch_size=train_params['batch_size'], 
                    num_epochs=train_params['eval_every'],
                    shuffle=True,
                    seed=42)

dev_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"ids": dev_x, "ns": dev_ns},
                    y=dev_y,
                    batch_size=128,
                    num_epochs=1,
                    shuffle=False)

## Train

In [19]:
for _ in range(train_params['total_epochs'] // train_params['eval_every']):
    model.train(input_fn=train_input_fn)
    model.evaluate(input_fn=dev_input_fn, name="dev")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tf_bow_sst_20180720-0205/model.ckpt.
INFO:tensorflow:loss = 2.34767, step = 1
INFO:tensorflow:global_step/sec: 170.282
INFO:tensorflow:loss = 4.27744, step = 101 (0.588 sec)
INFO:tensorflow:global_step/sec: 229.278
INFO:tensorflow:loss = 2.91387, step = 201 (0.436 sec)
INFO:tensorflow:global_step/sec: 232.748
INFO:tensorflow:loss = 1.96126, step = 301 (0.429 sec)
INFO:tensorflow:global_step/sec: 230.409
INFO:tensorflow:loss = 1.36261, step = 401 (0.434 sec)
INFO:tensorflow:global_step/sec: 232.398
INFO:tensorflow:loss = 1.53781, step = 501 (0.430 sec)
INFO:tensorflow:global_step/sec: 232.659
INFO:tensorflow:loss = 1.84371, step = 601 (0.430 sec)
INFO:tensorflow:global_step/sec: 232.868
INFO:tensorflow:l

INFO:tensorflow:global_step/sec: 231.491
INFO:tensorflow:loss = 1.2152, step = 8201 (0.432 sec)
INFO:tensorflow:global_step/sec: 230.843
INFO:tensorflow:loss = 1.01991, step = 8301 (0.433 sec)
INFO:tensorflow:global_step/sec: 234.603
INFO:tensorflow:loss = 0.984962, step = 8401 (0.426 sec)
INFO:tensorflow:global_step/sec: 235.774
INFO:tensorflow:loss = 0.789079, step = 8501 (0.424 sec)
INFO:tensorflow:global_step/sec: 228.807
INFO:tensorflow:loss = 1.13774, step = 8601 (0.437 sec)
INFO:tensorflow:global_step/sec: 234.026
INFO:tensorflow:loss = 1.00814, step = 8701 (0.427 sec)
INFO:tensorflow:global_step/sec: 232.056
INFO:tensorflow:loss = 1.29302, step = 8801 (0.431 sec)
INFO:tensorflow:global_step/sec: 234.918
INFO:tensorflow:loss = 1.05545, step = 8901 (0.426 sec)
INFO:tensorflow:global_step/sec: 229.271
INFO:tensorflow:loss = 1.13657, step = 9001 (0.436 sec)
INFO:tensorflow:global_step/sec: 231.828
INFO:tensorflow:loss = 1.02287, step = 9101 (0.431 sec)
INFO:tensorflow:global_step/s

INFO:tensorflow:global_step/sec: 226.149
INFO:tensorflow:loss = 1.22973, step = 16601 (0.442 sec)
INFO:tensorflow:global_step/sec: 232.401
INFO:tensorflow:loss = 0.819279, step = 16701 (0.430 sec)
INFO:tensorflow:global_step/sec: 231.903
INFO:tensorflow:loss = 1.12577, step = 16801 (0.431 sec)
INFO:tensorflow:global_step/sec: 229.072
INFO:tensorflow:loss = 0.867065, step = 16901 (0.437 sec)
INFO:tensorflow:global_step/sec: 233.307
INFO:tensorflow:loss = 0.957752, step = 17001 (0.428 sec)
INFO:tensorflow:global_step/sec: 227.801
INFO:tensorflow:loss = 0.866585, step = 17101 (0.439 sec)
INFO:tensorflow:global_step/sec: 231.556
INFO:tensorflow:loss = 0.884358, step = 17201 (0.432 sec)
INFO:tensorflow:global_step/sec: 226.957
INFO:tensorflow:loss = 0.972071, step = 17301 (0.440 sec)
INFO:tensorflow:global_step/sec: 226.804
INFO:tensorflow:loss = 0.887286, step = 17401 (0.441 sec)
INFO:tensorflow:global_step/sec: 229.645
INFO:tensorflow:loss = 1.00771, step = 17501 (0.435 sec)
INFO:tensorfl

INFO:tensorflow:global_step/sec: 228.373
INFO:tensorflow:loss = 0.889596, step = 23818 (0.438 sec)
INFO:tensorflow:global_step/sec: 228.814
INFO:tensorflow:loss = 0.923424, step = 23918 (0.437 sec)
INFO:tensorflow:global_step/sec: 231.619
INFO:tensorflow:loss = 1.21544, step = 24018 (0.432 sec)
INFO:tensorflow:global_step/sec: 230.019
INFO:tensorflow:loss = 1.04959, step = 24118 (0.435 sec)
INFO:tensorflow:global_step/sec: 229.47
INFO:tensorflow:loss = 0.831293, step = 24218 (0.436 sec)
INFO:tensorflow:global_step/sec: 225.239
INFO:tensorflow:loss = 1.25928, step = 24318 (0.444 sec)
INFO:tensorflow:global_step/sec: 227.935
INFO:tensorflow:loss = 0.978589, step = 24418 (0.439 sec)
INFO:tensorflow:global_step/sec: 230.227
INFO:tensorflow:loss = 0.821632, step = 24518 (0.434 sec)
INFO:tensorflow:global_step/sec: 230.167
INFO:tensorflow:loss = 0.78908, step = 24618 (0.434 sec)
INFO:tensorflow:global_step/sec: 228.474
INFO:tensorflow:loss = 0.892836, step = 24718 (0.438 sec)
INFO:tensorflow

INFO:tensorflow:loss = 0.762017, step = 32118 (0.435 sec)
INFO:tensorflow:global_step/sec: 230.553
INFO:tensorflow:loss = 1.3915, step = 32218 (0.434 sec)
INFO:tensorflow:global_step/sec: 229.119
INFO:tensorflow:loss = 0.976676, step = 32318 (0.436 sec)
INFO:tensorflow:global_step/sec: 229.085
INFO:tensorflow:loss = 1.10003, step = 32418 (0.437 sec)
INFO:tensorflow:global_step/sec: 231.963
INFO:tensorflow:loss = 0.791808, step = 32518 (0.431 sec)
INFO:tensorflow:global_step/sec: 228.954
INFO:tensorflow:loss = 0.740727, step = 32618 (0.437 sec)
INFO:tensorflow:global_step/sec: 227.262
INFO:tensorflow:loss = 0.801747, step = 32718 (0.440 sec)
INFO:tensorflow:global_step/sec: 218.683
INFO:tensorflow:loss = 1.22094, step = 32818 (0.457 sec)
INFO:tensorflow:global_step/sec: 228.613
INFO:tensorflow:loss = 0.828115, step = 32918 (0.437 sec)
INFO:tensorflow:global_step/sec: 230.922
INFO:tensorflow:loss = 0.835618, step = 33018 (0.433 sec)
INFO:tensorflow:global_step/sec: 229.783
INFO:tensorflo

...

INFO:tensorflow:loss = 0.778594, step = 181737 (0.439 sec)
INFO:tensorflow:global_step/sec: 224.301
INFO:tensorflow:loss = 0.652194, step = 181837 (0.446 sec)
INFO:tensorflow:global_step/sec: 227.782
INFO:tensorflow:loss = 0.654212, step = 181937 (0.439 sec)
INFO:tensorflow:global_step/sec: 227.076
INFO:tensorflow:loss = 1.15913, step = 182037 (0.440 sec)
INFO:tensorflow:global_step/sec: 228.837
INFO:tensorflow:loss = 0.797696, step = 182137 (0.437 sec)
INFO:tensorflow:global_step/sec: 229.673
INFO:tensorflow:loss = 0.826012, step = 182237 (0.435 sec)
INFO:tensorflow:global_step/sec: 230.352
INFO:tensorflow:loss = 0.638295, step = 182337 (0.434 sec)
INFO:tensorflow:global_step/sec: 230.997
INFO:tensorflow:loss = 0.755446, step = 182437 (0.433 sec)
INFO:tensorflow:global_step/sec: 227.299
INFO:tensorflow:loss = 0.849203, step = 182537 (0.440 sec)
INFO:tensorflow:global_step/sec: 227.457
INFO:tensorflow:loss = 0.924424, step = 182637 (0.440 sec)
INFO:tensorflow:global_step/sec: 226.621
I

INFO:tensorflow:global_step/sec: 225.897
INFO:tensorflow:loss = 0.826705, step = 190037 (0.443 sec)
INFO:tensorflow:global_step/sec: 227.199
INFO:tensorflow:loss = 0.909878, step = 190137 (0.440 sec)
INFO:tensorflow:global_step/sec: 226.623
INFO:tensorflow:loss = 1.00222, step = 190237 (0.441 sec)
INFO:tensorflow:global_step/sec: 226.8
INFO:tensorflow:loss = 1.12113, step = 190337 (0.441 sec)
INFO:tensorflow:global_step/sec: 225.128
INFO:tensorflow:loss = 1.05632, step = 190437 (0.444 sec)
INFO:tensorflow:global_step/sec: 225.844
INFO:tensorflow:loss = 0.697945, step = 190537 (0.443 sec)
INFO:tensorflow:global_step/sec: 226.348
INFO:tensorflow:loss = 0.684678, step = 190637 (0.442 sec)
INFO:tensorflow:global_step/sec: 227.667
INFO:tensorflow:loss = 0.912812, step = 190737 (0.439 sec)
INFO:tensorflow:global_step/sec: 229.385
INFO:tensorflow:loss = 0.674652, step = 190837 (0.436 sec)
INFO:tensorflow:global_step/sec: 223.693
INFO:tensorflow:loss = 0.837965, step = 190937 (0.447 sec)
INFO:

INFO:tensorflow:global_step/sec: 226.146
INFO:tensorflow:loss = 0.977258, step = 197154 (0.442 sec)
INFO:tensorflow:global_step/sec: 227.222
INFO:tensorflow:loss = 0.753467, step = 197254 (0.440 sec)
INFO:tensorflow:global_step/sec: 227.745
INFO:tensorflow:loss = 1.07567, step = 197354 (0.439 sec)
INFO:tensorflow:global_step/sec: 224.914
INFO:tensorflow:loss = 1.01639, step = 197454 (0.445 sec)
INFO:tensorflow:global_step/sec: 226.243
INFO:tensorflow:loss = 0.978849, step = 197554 (0.442 sec)
INFO:tensorflow:global_step/sec: 227.315
INFO:tensorflow:loss = 0.760396, step = 197654 (0.440 sec)
INFO:tensorflow:global_step/sec: 225.013
INFO:tensorflow:loss = 0.712556, step = 197754 (0.444 sec)
INFO:tensorflow:global_step/sec: 227.528
INFO:tensorflow:loss = 0.99439, step = 197854 (0.439 sec)
INFO:tensorflow:global_step/sec: 226.264
INFO:tensorflow:loss = 0.925445, step = 197954 (0.442 sec)
INFO:tensorflow:global_step/sec: 224.307
INFO:tensorflow:loss = 0.592587, step = 198054 (0.446 sec)
INF

INFO:tensorflow:loss = 0.90708, step = 205354 (0.437 sec)
INFO:tensorflow:global_step/sec: 228.892
INFO:tensorflow:loss = 0.90915, step = 205454 (0.437 sec)
INFO:tensorflow:global_step/sec: 226.954
INFO:tensorflow:loss = 1.14467, step = 205554 (0.441 sec)
INFO:tensorflow:global_step/sec: 227.594
INFO:tensorflow:loss = 1.13225, step = 205654 (0.440 sec)
INFO:tensorflow:global_step/sec: 227.332
INFO:tensorflow:loss = 0.770344, step = 205754 (0.440 sec)
INFO:tensorflow:global_step/sec: 229.282
INFO:tensorflow:loss = 0.569612, step = 205854 (0.436 sec)
INFO:tensorflow:global_step/sec: 228.908
INFO:tensorflow:loss = 1.13202, step = 205954 (0.437 sec)
INFO:tensorflow:global_step/sec: 230.831
INFO:tensorflow:loss = 0.873574, step = 206054 (0.433 sec)
INFO:tensorflow:global_step/sec: 228.71
INFO:tensorflow:loss = 0.756247, step = 206154 (0.437 sec)
INFO:tensorflow:global_step/sec: 227.56
INFO:tensorflow:loss = 1.03098, step = 206254 (0.440 sec)
INFO:tensorflow:global_step/sec: 227.696
INFO:ten

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-20-02:21:27
INFO:tensorflow:Saving dict for global step 213170: accuracy = 0.735263, cross_entropy_loss = 0.729775, global_step = 213170, loss = 0.95296


## Test

In [20]:
test_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"ids": test_x, "ns": test_ns},
                    y=test_y,
                    batch_size=128,
                    num_epochs=1,
                    shuffle=False)

eval_metrics = model.evaluate(input_fn=test_input_fn, name="test")
print("Accuracy on test set: {:.02%}".format(eval_metrics['accuracy']))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-20-02:21:27
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20180720-0205/model.ckpt-213170
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-20-02:21:33
INFO:tensorflow:Saving dict for global step 213170: accuracy = 0.719176, cross_entropy_loss = 0.764037, global_step = 213170, loss = 0.989033
Accuracy on test set: 71.92%
